In [29]:
import pymysql
import pandas as pd
class DB_Bot:
    symbol = ["BTC/USDT", "ETH/USDT", "XRP/USDT", "ETC/USDT", "ADA/USDT"]
    def __init__(self,coin_name, since = "2020-01-01 00:00:00"):
        self.coin_name = coin_name
        self.since = since
    
    def connect_db(self):
        host = "127.0.0.1"
        port = 3306
        username = "root"
        database = "CoinData"
        password = "825582qaz"
        try:
            con = pymysql.connect(host=host, user=username, password=password,
                            db=database, charset='utf8') # 한글처리 (charset = 'utf8')
        except:
            print(">> connection 실패 ")
            return False

        return con
    
    def GetData(self):
        con = self.connect_db()
        cur = con.cursor()
        sql = "SELECT * FROM CoinData."+str(self.coin_name)+" where timestamp >= " +"'"+ str(self.since)+"'"+";"
        
        cur.execute(sql)
        data = cur.fetchall()
        
        cur.close()
        con.close()
        col = ["datetime", "open", "high", "low", "close", "volume"]
        data = pd.DataFrame(data,columns = col)
        return data
    
    # return datetime obj
    def GetSetDate(self):
        con = self.connect_db()
        cur = con.cursor()
        sql = "SELECT * FROM CoinData.BTC_USDT_1m ORDER BY timestamp DESC LIMIT 1;"
        cur.execute(sql)
        data = cur.fetchall()
        return data[0][0]

In [30]:
from datetime import timedelta
symbol = "BTC_USDT_1m"
timeframe = str(symbol[9:])
t_num = int(timeframe.replace("m","").replace("h","").replace("d",""))
t_str = ''.join([c for c in timeframe if not c.isdigit()])
data_obj = DB_Bot(symbol)
now_date = data_obj.GetSetDate() 
if t_str == "m":
    previous_time = ((now_date - timedelta(minutes=500 * t_num))).strftime('%Y-%m-%d %H:%M:%S')
elif t_str == "h":
    previous_time = ((now_date - timedelta(hours=500 * t_num))).strftime('%Y-%m-%d %H:%M:%S')
elif t_str == "d":
    previous_time = ((now_date - timedelta(days=500 * t_num))).strftime('%Y-%m-%d %H:%M:%S')

data_obj.since = previous_time
data = data_obj.GetData()
data

,datetime,open,high,low,close,volume
0,2023-06-20 06:42:00,26804.47,26804.47,26798.64,26798.64,7.06467
1,2023-06-20 06:43:00,26798.65,26798.65,26790.01,26798.63,11.96509
2,2023-06-20 06:44:00,26798.64,26798.64,26792.33,26792.33,7.84219
3,2023-06-20 06:45:00,26792.33,26797.57,26792.33,26797.56,7.95502
4,2023-06-20 06:46:00,26797.55,26820.64,26797.55,26819.11,38.28369
...,...,...,...,...,...,...
496,2023-06-20 14:58:00,26772.73,26772.74,26768.00,26768.01,11.96060
497,2023-06-20 14:59:00,26768.01,26768.01,26768.00,26768.01,0.35861
498,2023-06-20 15:00:00,26780.00,26780.01,26770.12,26770.12,1.57330
499,2023-06-20 15:01:00,26783.99,26787.90,26783.99,26787.90,0.54181
